In [ ]:
import os
import subprocess
import time
import bittensor as bt

pow_timeout = 30
pow_min_difficulty = 7
pow_max_difficulty = 12

success_weight = 1
difficulty_weight = 1
time_elapsed_weight = 0.3
failed_penalty_weight = 0.4
allocation_weight = 0.21

max_score_challenge = 100 * (success_weight + difficulty_weight + time_elapsed_weight)
max_score = max_score_challenge + 100 * allocation_weight

failed_penalty_exp = 1.5

# Define the stock python reward function

def normalize(val, min_value, max_value):
    return (val - min_value) / (max_value - min_value)


def prevent_none(val):
    return 0 if not val else val


def percent(a, b):
    if b == 0:
        return 0
    return (a / b) * 100


def percent_yield(a, b):
    if a == 0:
        return 100
    return ((b - a) / b) * 100


def calc_score(
    response,
    hotkey,
    allocated_hotkeys,
    penalized_hotkeys,
    validator_hotkeys,
    jolt_params,
    i
):
    try:
        challenge_attempts = prevent_none(response.get("challenge_attempts", 1))
        challenge_successes = prevent_none(response.get("challenge_successes", 0))
        last_20_challenge_failed = prevent_none(
            response.get("last_20_challenge_failed", 0)
        )
        challenge_elapsed_time_avg = prevent_none(
            response.get("challenge_elapsed_time_avg", pow_timeout)
        )
        challenge_difficulty_avg = prevent_none(
            response.get("challenge_difficulty_avg", pow_min_difficulty)
        )
        has_docker = response.get("has_docker", False)

        assert challenge_attempts == jolt_params["challenge_attempts"][i], f"challenge_attempts: {challenge_attempts} != {jolt_params['challenge_attempts'][i]}"
        assert challenge_successes == jolt_params["challenge_successes"][i], f"challenge_successes: {challenge_successes} != {jolt_params['challenge_successes'][i]}"
        assert last_20_challenge_failed == jolt_params["last_20_challenge_failed"][i], f"last_20_challenge_failed: {last_20_challenge_failed} != {jolt_params['last_20_challenge_failed'][i]}"
        assert challenge_elapsed_time_avg == jolt_params["challenge_elapsed_time_avg"][i], f"challenge_elapsed_time_avg: {challenge_elapsed_time_avg} != {jolt_params['challenge_elapsed_time_avg'][i]}"
        assert challenge_difficulty_avg == jolt_params["challenge_difficulty_avg"][i], f"challenge_difficulty_avg: {challenge_difficulty_avg} != {jolt_params['challenge_difficulty_avg'][i]}"
        assert has_docker == jolt_params["has_docker"][i], f"has_docker: {has_docker} != {jolt_params['has_docker'][i]}"

        difficulty_val = max(
            min(challenge_difficulty_avg, pow_max_difficulty),
            pow_min_difficulty,
        )
        difficulty_modifier = percent(difficulty_val, pow_max_difficulty)

        difficulty = difficulty_modifier * difficulty_weight
        successes_ratio = percent(challenge_successes, challenge_attempts)
        successes = successes_ratio * success_weight

        time_elapsed_modifier = percent_yield(
            challenge_elapsed_time_avg, pow_timeout
        )
        time_elapsed = time_elapsed_modifier * time_elapsed_weight

        last_20_challenge_failed_modifier = percent(
            last_20_challenge_failed, 20
        )
        failed_penalty = (
            failed_penalty_weight
            * (last_20_challenge_failed_modifier / 100) ** failed_penalty_exp
            * 100
        )

        allocation_score = difficulty_modifier * allocation_weight
        allocation_status = hotkey in allocated_hotkeys

        assert max_score_challenge == 100 * (
            success_weight + difficulty_weight + time_elapsed_weight
        )
        max_score_allocation = 100 * allocation_weight
        assert max_score == max_score_challenge + max_score_allocation
        final_score = difficulty + successes + time_elapsed - failed_penalty

        penalty = not (has_docker)

        if allocation_status:
            final_score = (
                max_score_challenge * (1 - allocation_weight) + allocation_score
            )
        else:
            final_score = difficulty + successes + time_elapsed - failed_penalty
            if penalty:
                final_score = final_score / 2

        if (
            last_20_challenge_failed >= 19 or challenge_successes == 0
        ) and not allocation_status:
            print("Returning 0 due to high failure rate or no successes")
            return 0

        if hotkey in penalized_hotkeys:
            penalty_count = penalized_hotkeys.count(hotkey)
            half_validators = len(validator_hotkeys) / 2
            assert half_validators == jolt_params["half_validators"], f"half_validators: {half_validators} != {jolt_params['half_validators']}"

            if penalty_count >= half_validators:
                final_score = 0
            else:
                penalty_multiplier = max(1 - (penalty_count / half_validators), 0)
                final_score *= penalty_multiplier

        final_score = max(0, final_score)

        normalized_score = normalize(final_score, 0, max_score)

        return normalized_score
    except Exception as e:
        bt.logging.error(
            f"An error occurred while calculating score for the following hotkey - {hotkey}: {e}"
        )
        return 0


runtimes = [0]
time_now = time.time()
existing_dir = os.getcwd()
os.chdir(
    "../../neurons/deployment_layer/model_1d60d545b7c5123fd60524dcbaf57081ca7dc4a9ec36c892927a3153328d17c0"
)
subprocess.run(["cargo", "build", "--release"], check=True)
runtimes.append(time.time() - time_now)
time_now = time.time()

In [ ]:
from typing import List, Dict, Any
import random
import json
import subprocess
import numpy as np
from rich.console import Console
from rich.table import Table
import matplotlib.pyplot as plt

BATCH_SIZE = 256
VALIDATOR_HOTKEYS = [
    "second", "third", "fourth", "fifth", "sixth",
    "seventh", "eighth", "ninth", "tenth"
]

def generate_input(pow_min_difficulty: float, pow_max_difficulty: float, pow_timeout: float) -> Dict[str, Any]:
    allocated_hotkeys = VALIDATOR_HOTKEYS.copy()
    penalized_hotkeys = VALIDATOR_HOTKEYS.copy()
    allocated_hotkeys.extend(['dummy_hotkey'] * random.randint(0, 3))
    penalized_hotkeys.extend(['dummy_hotkey'] * random.randint(0, 3))
    attempts = random.randint(1, 7)
    return {
        "challenge_attempts": attempts,
        "challenge_successes": random.randint(0, attempts),
        "last_20_challenge_failed": random.randint(0, 20),
        "challenge_elapsed_time_avg": random.uniform(0.001, pow_timeout),
        "challenge_difficulty_avg": random.uniform(pow_min_difficulty, pow_max_difficulty),
        "has_docker": random.choice([True, False]),
        "allocated_hotkeys": allocated_hotkeys,
        "penalized_hotkeys": penalized_hotkeys
    }

def generate_inputs(batch_size: int, pow_min_difficulty: float, pow_max_difficulty: float, pow_timeout: float) -> List[Dict[str, Any]]:
    return [generate_input(pow_min_difficulty, pow_max_difficulty, pow_timeout) for _ in range(batch_size)]

def prepare_jolt_inputs(inputs: List[Dict[str, Any]], params: Dict[str, Any]) -> Dict[str, Any]:
    return {
        **params,
        "challenge_attempts": [input_data["challenge_attempts"] for input_data in inputs],
        "challenge_successes": [input_data["challenge_successes"] for input_data in inputs],
        "last_20_challenge_failed": [input_data["last_20_challenge_failed"] for input_data in inputs],
        "challenge_elapsed_time_avg": [input_data["challenge_elapsed_time_avg"] for input_data in inputs],
        "challenge_difficulty_avg": [input_data["challenge_difficulty_avg"] for input_data in inputs],
        "has_docker": [input_data["has_docker"] for input_data in inputs],
        "allocated_hotkey": ["dummy_hotkey" in input_data["allocated_hotkeys"] for input_data in inputs],
        "penalized_hotkey_count": [input_data["penalized_hotkeys"].count("dummy_hotkey") for input_data in inputs],
        "half_validators": len(VALIDATOR_HOTKEYS) / 2,
        "nonce": random.getrandbits(128),
    }

def run_original_calc_score(inputs: List[Dict[str, Any]], jolt_inputs: Dict[str, Any], calc_score_func) -> List[float]:
    return [
        calc_score_func(
            response=input_data,
            hotkey="dummy_hotkey",
            allocated_hotkeys=input_data["allocated_hotkeys"],
            penalized_hotkeys=input_data["penalized_hotkeys"],
            validator_hotkeys=VALIDATOR_HOTKEYS,
            jolt_params=jolt_inputs,
            i=i
        ) for i, input_data in enumerate(inputs)
    ]

def run_jolt_calc_score(jolt_inputs: Dict[str, Any]) -> List[float]:
    with open("input.json", "w") as outfile:
        json.dump(jolt_inputs, outfile)

    subprocess.run(
        ["cargo", "run", "--release", "prove", "--input", "input.json", "--output", "output.json", "--proof", "proof.bin"],
        check=True
    )

    with open("output.json", "r") as infile:
        return json.load(infile)

def compare_scores(inputs: List[Dict[str, Any]], original_scores: List[float], jolt_scores: List[float]) -> None:
    table = Table(title="Score Comparison")
    columns = [
        ("UID", "right", "cyan"),
        ("Challenge Attempts", "right", "blue"),
        ("Challenge Successes", "right", "blue"),
        ("Last 20 Failed", "right", "blue"),
        ("Elapsed Time Avg", "right", "blue"),
        ("Difficulty Avg", "right", "blue"),
        ("Has Docker", "center", "blue"),
        ("Allocated", "center", "blue"),
        ("Penalized", "center", "blue"),
        ("Original Score", "right", "magenta"),
        ("Jolt Score", "right", "green"),
        ("Difference", "right", "red")
    ]
    for col, justify, style in columns:
        table.add_column(col, justify=justify, style=style)

    red_diff_count = 0
    total_diff = 0.0

    for i, (input_data, orig, jolt) in enumerate(zip(inputs, original_scores, jolt_scores)):
        diff = np.abs(np.float32(jolt) - np.float32(orig))
        total_diff += diff
        diff_color = "red" if diff > 1e-6 else "green"
        red_diff_count += diff_color == "red"
        table.add_row(
            f"{i}",
            f"{input_data['challenge_attempts']}",
            f"{input_data['challenge_successes']}",
            f"{input_data['last_20_challenge_failed']}",
            f"{input_data['challenge_elapsed_time_avg']:.4f}",
            f"{input_data['challenge_difficulty_avg']:.4f}",
            "✓" if input_data['has_docker'] else "✗",
            "✓" if "dummy_hotkey" in input_data['allocated_hotkeys'] else "✗",
            "✓" if "dummy_hotkey" in input_data['penalized_hotkeys'] else "✗",
            f"{orig:.6f}",
            f"{jolt:.6f}",
            f"[{diff_color}]{diff:+.6f}[/{diff_color}]"
        )

    total_count = len(original_scores)
    red_percentage = (red_diff_count / total_count) * 100
    avg_diff = total_diff / total_count
    table.add_row("", "", "", "", "", "", "", "", "", "", "", "")
    table.add_row("", "", "", "", "", "", "", "", "", "", "High Loss %:", f"[red]{red_percentage:.2f}%[/red]")
    table.add_row("", "", "", "", "", "", "", "", "", "", "Avg Difference:", f"[yellow]{avg_diff:.6f}[/yellow]")

    console = Console()
    console.width = 140
    console.print(table)

def plot_scores(inputs: List[Dict[str, Any]], original_scores: List[float], jolt_scores: List[float]) -> None:
    def sort_and_unzip(scores):
        return zip(*sorted(enumerate(scores), key=lambda x: x[1]))

    original_indices, original_sorted_scores = sort_and_unzip(original_scores)
    jolt_indices, jolt_sorted_scores = sort_and_unzip(jolt_scores)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    for ax, indices, sorted_scores, title in [
        (ax1, original_indices, original_sorted_scores, 'Original Scores (Sorted)'),
        (ax2, jolt_indices, jolt_sorted_scores, 'Jolt Scores (Sorted)')
    ]:
        scatter = ax.scatter(
            range(len(sorted_scores)),
            sorted_scores,
            c=[inputs[i]["challenge_difficulty_avg"] for i in indices],
            cmap="viridis",
            alpha=0.6,
        )
        plt.colorbar(scatter, ax=ax, label='Challenge Difficulty Avg')
        ax.set_title(title)
        ax.set_xlabel('Index (Sorted by Score)')
        ax.set_ylabel('Score')

    plt.tight_layout()
    plt.show()

def main():
    params = {
        "success_weight": success_weight,
        "difficulty_weight": difficulty_weight,
        "time_elapsed_weight": time_elapsed_weight,
        "failed_penalty_weight": failed_penalty_weight,
        "allocation_weight": allocation_weight,
        "pow_min_difficulty": pow_min_difficulty,
        "pow_max_difficulty": pow_max_difficulty,
        "pow_timeout": pow_timeout,
        "max_score_challenge": max_score_challenge,
        "max_score": max_score,
        "failed_penalty_exp": failed_penalty_exp,
    }

    inputs = generate_inputs(BATCH_SIZE, params["pow_min_difficulty"], params["pow_max_difficulty"], params["pow_timeout"])
    jolt_inputs = prepare_jolt_inputs(inputs, params)

    original_scores = run_original_calc_score(inputs, jolt_inputs, calc_score)
    jolt_scores = run_jolt_calc_score(jolt_inputs)

    compare_scores(inputs, original_scores, jolt_scores)
    plot_scores(inputs, original_scores, jolt_scores)

if __name__ == "__main__":
    main()